In [1]:
import pandas as pd
merged_df = pd.read_csv('Merged_DatasetV2.csv')
merged_df.head()

/var/folders/rf/qbnw5dmn5dq1gms0pls5p4b40000gn/T/ipykernel_20447/2551388421.py:2: DtypeWarning: Columns (4,5,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_df = pd.read_csv('Merged_DatasetV2.csv')


,Unnamed: 0,id,type,name,city,state,county,country,lifespan_years,status,services_offered
0,0,f9a1fd95-4c6d-43a6-bb31-4ee8d5a44a8b,Studio,Columbia (CBS) Studio ‘B’,Test Restaurant,New York,NaN,US,NaN,Active,NaN
1,1,42e9fa7c-a981-44f9-943c-bf39a8c6ce8d,Venue,New Theatre Oxford,Victoria House Hotel,England,Oxfordshire,GB,NaN,Active,NaN
2,2,34e7351d-715d-4b20-a5d6-3c0bd717eb1d,Religious building,Santuario della Beata Vergine di Ariadello,Genivolta,NaN,NaN,IT,359.0,Active,NaN
3,3,0ad32768-514b-410f-9d5a-40ebb0c3373c,Studio,ROKU-st,Tenso Jinja,NaN,NaN,JP,NaN,Active,NaN
4,4,0b050655-48da-4be7-b5de-7a3ee7370b9e,Venue,Rätschenmühle,Geislingen an der Steige,NaN,NaN,DE,NaN,Active,NaN


In [2]:
# Drop rows where 'services_offered' is null
merged_df = merged_df.dropna(subset=['services_offered'])

# Drop the specified columns
merged_df = merged_df.drop(columns=['Unnamed: 0', 'id', 'name'])

# Display the updated dataframe
merged_df.head()


,type,city,state,county,country,lifespan_years,status,services_offered
15520,Other,Los Jarales,NaN,NaN,VE,NaN,Active,recorded at
15521,Venue,Plasky,Brussels Capital,Bruxelles-Capitale,BE,62.0,Active,recorded at
15522,Other,Dunluce Castle,Northern Ireland,Causeway Coast and Glens,GB,NaN,Active,recorded at
15524,Studio,Saint Johns Eastern Orthodox Church,Tennessee,NaN,US,5.0,Ended,engineered at
15525,Venue,Shimokitazawa Eki,NaN,NaN,JP,34.0,Active,recorded at


In [3]:
# Split the 'services_offered' column by commas and explode the dataframe
merged_df['services_offered'] = merged_df['services_offered'].str.split(', ')
merged_df = merged_df.explode('services_offered')

In [4]:
def impute_df(df):
    # Fill non-numeric columns with "Missing"
    cols_to_fill = df.columns.difference(['lifespan_years'])
    df[cols_to_fill] = df[cols_to_fill].fillna("Missing")
    
    # Create missing indicator for 'lifespan_years'
    df['lifespan_years_missing'] = df['lifespan_years'].isnull().astype(int)
    
    # Impute missing values in 'lifespan_years' with the median value
    median_val = df['lifespan_years'].median()
    df['lifespan_years'] = df['lifespan_years'].fillna(median_val)
    
    return df

In [5]:
from sklearn.model_selection import train_test_split

# Perform train-test split with stratification on 'services_offered'
train_df, test_df = train_test_split(merged_df, test_size=0.2, stratify=merged_df['services_offered'], random_state=42)


train_df = impute_df(train_df)
test_df = impute_df(test_df)

In [9]:
# Exclude both "services_offered" (target) and "lifespan_years" from dummy encoding.
X_train = pd.get_dummies(train_df.drop(columns=["services_offered", "lifespan_years"]), drop_first=True)

# Add "lifespan_years" back as a numeric feature.
X_train["lifespan_years"] = train_df["lifespan_years"]

X_train.head()

,lifespan_years_missing,type_Club,type_Concert hall / Theatre,type_Educational institution,type_Festival stage,type_Indoor arena,type_Missing,type_Other,type_Park,type_Pressing plant,...,country_TR,country_TW,country_UA,country_US,country_UY,country_VA,country_VE,country_ZA,status_Ended,lifespan_years
68980,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,43.0
19071,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,43.0
58597,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,43.0
62065,1,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,43.0
18253,0,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,59.0


In [14]:
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Transform training data (as before)
X_train = pd.get_dummies(train_df.drop(columns=["services_offered", "lifespan_years"]), drop_first=True)
X_train["lifespan_years"] = train_df["lifespan_years"]
X_train.columns = X_train.columns.astype(str).str.replace('[,\[\]<>]', '_', regex=True)

le = LabelEncoder()
y_train = le.fit_transform(train_df["services_offered"])

# Initialize and train the XGBoost Classifier
model = xgb.XGBClassifier(random_state=42)
model.fit(X_train, y_train)

# Transform the test data in the same way
X_test = pd.get_dummies(test_df.drop(columns=["services_offered", "lifespan_years"]), drop_first=True)
X_test["lifespan_years"] = test_df["lifespan_years"]
X_test.columns = X_test.columns.astype(str).str.replace('[,\[\]<>]', '_', regex=True)
# Align test features with training features
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

y_test = le.transform(test_df["services_offered"])

# Evaluate on test data
preds_test = model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, preds_test))

<>:8: SyntaxWarning: invalid escape sequence '\['
<>:20: SyntaxWarning: invalid escape sequence '\['
<>:8: SyntaxWarning: invalid escape sequence '\['
<>:20: SyntaxWarning: invalid escape sequence '\['
/var/folders/rf/qbnw5dmn5dq1gms0pls5p4b40000gn/T/ipykernel_20447/1547110777.py:8: SyntaxWarning: invalid escape sequence '\['
  X_train.columns = X_train.columns.astype(str).str.replace('[,\[\]<>]', '_', regex=True)
/var/folders/rf/qbnw5dmn5dq1gms0pls5p4b40000gn/T/ipykernel_20447/1547110777.py:20: SyntaxWarning: invalid escape sequence '\['
  X_test.columns = X_test.columns.astype(str).str.replace('[,\[\]<>]', '_', regex=True)


Test Accuracy: 0.6728646135967461
